In [1]:
from keras.models import Sequential , Model
from keras.layers import Dense , Conv2D , MaxPool2D , BatchNormalization , Reshape , Input , Flatten
from keras.layers.advanced_activations import LeakyReLU

from keras.optimizers import Adam


C:\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import os

import matplotlib as plt
import numpy as np

%matplotlib inline

In [13]:
WIDTH = 96
HEIGHT = 96
CHANNEL = 3

LATENT_DIM = 100 #latent variable z

BATCH_SIZE = 2
EPOCHS = 10

PATH = 'faces/'

In [14]:
load_index = 0

images_name = os.listdir(PATH)

IMAGES_COUNT = len(images_name)

In [34]:
def load_image(batch_size = BATCH_SIZE):
    global load_index
    
    images = []
    
    for i in range(batch_size):
        images.append(plt.image.imread(PATH + images_name[(load_index + i) % IMAGES_COUNT]))
    
    load_index += batch_size
    
    return np.array(images)/255

def write_image(epoch):
    plt.image.imsave('images/'+str(epoch)+'.jpg')

In [21]:
def generator():
    #sample from noise z
    model = Sequential(name='generator')
    
    model.add(Dense(units = 256 , input_dim = LATENT_DIM , activation='relu' , name='dense1'))
    model.add(BatchNormalization(name='batchnorm1'))
    #model.add(Dense(units = 512 , activation='relu' , name='dense2'))
    #model.add(BatchNormalization(name='batchnorm2'))
    model.add(Dense(units = 1024 , activation='relu' , name='dense3'))
    model.add(BatchNormalization(name='batchnorm3'))
    #model.add(Dense(units = 5000 , activation='relu' , name='dense4'))
    #model.add(BatchNormalization(name='batchnorm4'))
    #model.add(Dense(units = 10000 , activation='relu' , name='dense5'))
    #model.add(BatchNormalization(name='batchnorm5'))
    #model.add(Dense(units = 20000 , activation='relu' , name='dense6'))
    #model.add(BatchNormalization(name='batchnorm6'))
    model.add(Dense(units = WIDTH*HEIGHT*CHANNEL , activation='tanh' , name='dense7'))
    model.add(Reshape(target_shape=(WIDTH , HEIGHT , CHANNEL) , name='reshape1'))
    
    noise = Input(shape=(LATENT_DIM , ) , name='input1')
    image = model(noise)
    
    return Model(noise , image , name='generator_Model')

In [22]:
def discriminator():
    #input a image to discriminate real or fake
    model = Sequential(name='discriminator')
    
    model.add(Flatten(input_shape = (WIDTH,HEIGHT,CHANNEL) , name='flatten1'))
    #model.add(Dense(units=10000 , name='dense8'))
    #model.add(LeakyReLU(0.3 , name='leakyrelu1'))
    #model.add(Dense(units=5000 , name='dense9'))
    #model.add(LeakyReLU(0.3 , name='leakyrelu2'))
    model.add(Dense(units=1024 , name='dense10'))
    model.add(LeakyReLU(0.3 , name='leakyrelu3'))
    #model.add(Dense(units=512 , name='dense11'))
    #model.add(LeakyReLU(0.3 , name='leakyrelu4'))
    #model.add(Dense(units=128 , name='dense12'))
    #model.add(LeakyReLU(0.3 , name='leakyrelu5'))
    model.add(Dense(units=32 , name='dense13'))
    model.add(LeakyReLU(0.3 , name='leakyrelu6'))
    model.add(Dense(units=4 , name='dense14'))
    model.add(LeakyReLU(0.3 , name='leakyrelu7'))
    model.add(Dense(units=1 , activation='sigmoid' , name='dense15'))

    image = Input(shape=(WIDTH , HEIGHT , CHANNEL) , name='input1')
    validity = model(image)
    
    return Model(image , validity , name='discriminator_Model')

In [23]:
def combined_model(generator_i , discriminator_i):
    #生成器和判别器组合成整体
    z = Input(shape=(LATENT_DIM , ) , name='z')
    image = generator_i(z)
    validity = discriminator_i(image)
    
    return Model(z , validity , name='combined_model')

In [35]:
real_labels = np.ones(shape=(BATCH_SIZE , 1))
fake_labels = np.ones(shape=(BATCH_SIZE , 1))

generator_i = generator()
discriminator_i = discriminator()
discriminator_i.compile(optimizer=Adam(lr=0.002) , loss='binary_crossentropy')
discriminator_i.trainable = False

combined_model_i = combined_model(generator_i , discriminator_i)
combined_model_i.compile(optimizer=Adam(lr=0.002) , loss='binary_crossentropy')


In [36]:
for i in range(20000):
    noise = np.random.normal(size=(BATCH_SIZE , LATENT_DIM))

    real_image = load_image()
    #训练判别器
    fake_image = generator_i.predict(noise)

    real_loss = discriminator_i.train_on_batch(real_image , real_labels)
    fake_loss = discriminator_i.train_on_batch(fake_image , fake_labels)

    loss = (real_loss+fake_loss)/2

    #训练生成器
    noise2 = np.random.normal(size=(BATCH_SIZE , LATENT_DIM))
    generator_loss = combined_model_i.train_on_batch(noise2 , real_labels)
    
    #if i%10 == 0:
    #    print(loss , generator_loss)
    if i % 100 == 0:
        write_image(i)

C:\Anaconda3\lib\site-packages\keras\engine\training.py:478: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


TypeError: imsave() missing 1 required positional argument: 'arr'

In [31]:
combined_model_i.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
z (InputLayer)               (None, 100)               0         
_________________________________________________________________
generator_Model (Model)      (None, 96, 96, 3)         28633344  
_________________________________________________________________
discriminator_Model (Model)  (None, 1)                 28345513  
Total params: 56,978,857
Trainable params: 28,630,784
Non-trainable params: 28,348,073
_________________________________________________________________


TypeError: compile() missing 1 required positional argument: 'optimizer'

In [64]:
mom = Sequential()
mom.add(Dense(1,input_dim=2))

In [65]:
mom.trainable=False

In [85]:
mom(name='dd')

TypeError: __call__() missing 1 required positional argument: 'inputs'

In [86]:
Model(name='d')

In [ ]:
combined_model_i.summar